In [12]:
import urllib

download_path = "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt"
data = urllib.request.urlopen(download_path)

text_chars = ""

whitespace_replace_chars = ["\\n"]
remove_chars = ["'", '"', "-", ",", ":", "?", "!", ";"]

for line in data:
    line = str(line).lower()[2:]
    for replace_char in whitespace_replace_chars:
        line = line.replace(replace_char, " ")
    for remove_char in remove_chars:
        line = line.replace(remove_char, "")
    text_chars += line
text_chars = ' '.join(text_chars.split())

In [14]:
print(text_chars[:2000])

first citizen before we proceed any further hear me speak. all speak speak. first citizen you are all resolved rather to die than to famish all resolved. resolved. first citizen first you know caius marcius is chief enemy to the people. all we knowt we knowt. first citizen let us kill him and well have corn at our own price. ist a verdict all no more talking ont let it be done away away second citizen one word good citizens. first citizen we are accounted poor citizens the patricians good. what authority surfeits on would relieve us if they would yield us but the superfluity while it were wholesome we might guess they relieved us humanely but they think we are too dear the leanness that afflicts us the object of our misery is as an inventory to particularise their abundance our sufferance is a gain to them let us revenge this with our pikes ere we become rakes for the gods know i speak this in hunger for bread not in thirst for revenge. second citizen would you proceed especially again

In [15]:
print(len(text_chars))

1061639


In [40]:
import torch

def batches(data, batch_size=32):
    l = len(data)
    for i in range(0, l, batch_size):
        yield data[i:min(i + batch_size, l)]

def letter_to_index(letter):
    return ALL_LETTERS.find(letter)

def letter_to_tensor(letter):
    letter_tensor = torch.zeros(1, N_LETTERS)
    letter_tensor[0][letter_to_index(letter)] = 1
    return letter_tensor

def line_to_tensor(line):
    line_tensor = torch.zeros(len(line), 1, N_LETTERS)
    for i, letter in enumerate(line):
        line_tensor[i][0][letter_to_index(letter)] = 1
    return line_tensor

In [43]:
import string

ALL_LETTERS = string.ascii_letters + " .,;'"
N_LETTERS = len(ALL_LETTERS)

print(line_to_tensor(text_chars[:10]).shape) # torch.Size([10, 1, 57]) = BATCH_SIZE x 1 x N_LETTERS

data_lines = []
for batch in batches(data, batch_size=16):
    data_lines.append(batch)

torch.Size([10, 1, 57])


In [44]:
print(line_to_tensor(data_lines[54]).shape)

torch.Size([16, 1, 57])
